In [ ]:
# Read the file and perform exploratory  data analysis

In [38]:
import pandas as pd
cr = pd.read_csv("E:\DATA scientist\R\Creditcard\CreditRisk.csv")

In [39]:
cr.Self_Employed = cr.Self_Employed.fillna( "Yes")
cr.Married.fillna("Yes" , inplace= True)
cr.Gender = cr.Gender.fillna("Male")
cr.Dependents = cr.Dependents.fillna(0)
cr.LoanAmount  = cr.LoanAmount.fillna(  cr.LoanAmount.mean()   )
cr.Loan_Amount_Term  =cr.Loan_Amount_Term.fillna(cr.Loan_Amount_Term.mean())
cr.Credit_History  = cr.Credit_History.fillna( 0)

In [40]:
cr = cr.drop(["Loan_ID"] , axis = 1)

In [41]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [42]:
cr[cr.select_dtypes(include = ["object"]).columns] = cr[cr.select_dtypes(include = ["object"]).columns].apply(le.fit_transform)

In [43]:
cr.head(5)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1,0,0.0,0,0,5849,0.0,142.51153,360.0,1.0,2,1
1,1,1,1.0,0,0,4583,1508.0,128.00000,360.0,1.0,0,0
2,1,1,0.0,0,1,3000,0.0,66.00000,360.0,1.0,2,1
3,1,1,0.0,1,0,2583,2358.0,120.00000,360.0,1.0,2,1
4,1,0,0.0,0,0,6000,0.0,141.00000,360.0,1.0,2,1


In [44]:
cr.x = cr.iloc[:,1:-1]
cr.y = cr.iloc[:,-1]

<ipython-input-44-262ef4363961>:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  cr.x = cr.iloc[:,1:-1]
<ipython-input-44-262ef4363961>:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  cr.y = cr.iloc[:,-1]


In [ ]:
# Apply Random sampling on the data

In [45]:
from sklearn.model_selection import train_test_split
cr.x_train, cr.x_test, cr.y_train, cr.y_test = train_test_split(cr.x,cr.y,test_size = 0.2)

<ipython-input-45-d29d2e127533>:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  cr.x_train, cr.x_test, cr.y_train, cr.y_test = train_test_split(cr.x,cr.y,test_size = 0.2)


In [31]:
# Import decision tree and Grid Search

In [67]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
dt = DecisionTreeClassifier()

In [68]:
search_dict = {"criterion" :["gini" , "entropy"],
               "max_depth" : range(3,10),
               "min_samples_split" :range(25,50) }

In [ ]:
# We have set the range of maximum depth from 3 to 10  and range of minimum sample split 25 to 50. 
# Now let us fit the decision tree in Grid Search and train the data with decision tree.

In [69]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(  dt ,param_grid = search_dict)

In [70]:
grid.fit(cr.x_train , cr.y_train)

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(3, 10),
                         'min_samples_split': range(25, 50)})

In [71]:
grid.best_score_

0.809962436714029

In [ ]:
# We can see that the best grid accuracy is 80%. 

In [72]:
grid.best_params_

{'criterion': 'entropy', 'max_depth': 3, 'min_samples_split': 25}

In [ ]:
# The best grid parameters are: 'max_depth': 3, 'min_samples_split': 25
# Hence, let us set the paramaters and apply 10 fold cross validation of 10 folds.

In [79]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
dt = DecisionTreeClassifier(min_samples_split=25)
scores = cross_val_score(dt, cr.x_train , cr.y_train  , cv = 10 )
scores

array([0.73417722, 0.75949367, 0.81012658, 0.73417722, 0.74358974,
       0.74358974, 0.71794872, 0.75641026, 0.80769231, 0.74358974])

In [80]:
import numpy as np
np.mean(scores)

0.7550795196364817

In [ ]:
# The mean score is 75%.

In [81]:
from sklearn.metrics import confusion_matrix
dt.fit(cr.x_train,cr.y_train)

DecisionTreeClassifier(min_samples_split=25)

In [82]:
pred_dt= dt.predict(cr.x_test)

In [83]:
from sklearn.metrics import confusion_matrix
tab_dt_grid = confusion_matrix(pred_dt, cr.y_test)
tab_dt_grid

array([[ 25,  29],
       [ 25, 118]], dtype=int64)

In [84]:
accuracy = tab_dt_grid.diagonal().sum() * 100 / tab_dt_grid.sum()
accuracy

72.58883248730965

In [ ]:
# As we can see, the accuracy of the model is 72%.